거스름돈의 금액을 나타내는 숫자가 주어졌을 때, 거스름돈을 낼 수 있는 동전의 최소 개수가 몇 개인지 구하는 함수를 작성하라. 

예) 미국 동전이 있다고 가정해보자: 1, 5, 10, 25 센트

makeChange(1) = 1 (1)

makeChange(2) = 2 (5+1)

makehange(49) = 7 (25 + 10 + 10 + 1 + 1 + 1 + 1)

## 첫 번째 해법 찾기

첫 번째 해법부터 찾아보자. 최대/최소/가장 큰 경우의 수/가장 작은 경우의 수를 찾는 문제를 풀 때 좋은 기술 중 하나는 가능한 모든 조합들을 비교해 보는 것이다.